<a href="https://colab.research.google.com/github/Viktoriya89/doc_tex_source/blob/main/Caltech101_lessNetInfo_module4_VSergeyeva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projet Deep Image du module 4, en ré-utilisant le TP vu en cours.

In [45]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


On importe tous ce dont on aura besoin dans notre code.

In [46]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.utils as vutils
import torch.backends.cudnn as cudnn

from torch.utils.data import Subset
from torch.utils.data.sampler import SubsetRandomSampler

In [47]:
# Progrès
from tqdm import tqdm
from tqdm.autonotebook import tqdm
#from tqdm import tqdm_notebook as tqdm

#matpotlib 
#import matplotlib
import matplotlib.pyplot as plt

# Modules (torch, nn, F et optim)
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
from PIL import Image

Processing /root/.cache/pip/wheels/4f/0a/2a/7e3391063af230fac4b5fdb4cc93adcb1d99af325b623cea03/Pillow-4.0.0-cp37-cp37m-linux_x86_64.whl
ERROR: torchvision 0.9.1+cu101 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
ERROR: scikit-image 0.16.2 has requirement pillow>=4.3.0, but you'll have pillow 4.0.0 which is incompatible.
ERROR: bokeh 2.3.0 has requirement pillow>=7.1.0, but you'll have pillow 4.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.0
    Uninstalling Pillow-7.1.0:
      Successfully uninstalled Pillow-7.1.0


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


On re-utilise la fonction qui effectue l'apprentissage - train()

In [48]:
def set_parameter_requires_grad(requires_grad=True):
    for param in net.parameters():
        param.requires_grad = requires_grad

my_num_classes = 101

def train(nbEpochs=3):
    #print(net)
    
    criterion = nn.CrossEntropyLoss()
    for epoch in range(nbEpochs):  
        tloss = 0.0
        i=0
        t = tqdm(iter(trainloader), total=len(trainloader), dynamic_ncols=True)
        for data in t:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            tloss += loss.item()
        
        tloss=tloss/len(trainloader)
        #tbw.add_scalar("loss/train", tloss, epoch)
        print("train loss=",epoch, tloss) 
        
        tloss=0.0
        ok = 0
        tot = 0
        with torch.no_grad():
            t = tqdm(iter(testloader), total=len(testloader))
            for data in t:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                _, predicted = torch.max(outputs.data, 1)
                tot += labels.size(0)
                ok += (predicted == labels).sum().item()
                tloss += loss.item()
        
        tloss=tloss/len(testloader)
        #tbw.add_scalar("loss/test", tloss, epoch)
        print("test loss=",epoch, tloss)
               
        print('Accuracy in test', (100.0 * ok / tot))

On charge les modeles pré-entrainés tels comme ils sont.

In [49]:
from torchvision import models
net1 = models.resnet18(pretrained=True)
#print(net1)
# Derniere couche:
# (fc): Linear(in_features=512, out_features=1000, bias=True)

net2 = models.alexnet(pretrained=True)
#print(net2)
# Derniere couche:
# (6): Linear(in_features=4096, out_features=1000, bias=True)

net3 = models.squeezenet1_0(pretrained=True)
#print(net3)
# Derniere couche:
# (3): AdaptiveAvgPool2d(output_size=(1, 1))

net4 = models.vgg16(pretrained=True)
#print(net4)
# Derniere couche:
# (6): Linear(in_features=4096, out_features=1000, bias=True)

net5 = models.densenet161(pretrained=True)
#print(net5)
# Derniere couche:
# (classifier): Linear(in_features=2208, out_features=1000, bias=True)

net6 = models.inception_v3(pretrained=True)
#print(net6)
# Derniere couche x2!
# (AuxLogits): InceptionAux(
    #(conv0): BasicConv2d(
      #(conv): Conv2d(768, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      #(bn): BatchNorm2d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    #)
    #(conv1): BasicConv2d(
      #(conv): Conv2d(128, 768, kernel_size=(5, 5), stride=(1, 1), bias=False)
      #(bn): BatchNorm2d(768, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    #)
    #(fc): Linear(in_features=768, out_features=1000, bias=True)
# Et l'output habituel:
# (fc): Linear(in_features=2048, out_features=1000, bias=True)
# On doit modifier les 2 couches de sortie!

On definit un ensemble des reseaux avec les modeles de 1 à 5, tous sauf Inception_v3. Puis, Inception_v3 est à part, pour changer la taille de l'image d'entrée.

In [50]:
#nets = [net1, net2, net3, net4, net5]
nets = [net2]
nets2 = [net6]

On fixe les poids des modeles. Nous allons en suite modifier la dernière couche de chaque modele pour adapter à notre nombre des sorties = 101.

In [51]:
# Fixons d'abord les poids du réseau :
for net in nets:
  set_parameter_requires_grad(False)

for net in nets2:
  set_parameter_requires_grad(False)

# On regarde les examples sur:
#https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

# Pour avoir 101 classes et non 1000 en sortie :
# modif net1, ResNet18
num_ftrs = net1.fc.in_features
net1.fc = nn.Linear(num_ftrs, my_num_classes)
#print(net1)
# Derniere couche
# (fc): Linear(in_features=512, out_features=101, bias=True)

# modif net2, alexnet
#(12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False) -> remove this MaxPool2d or change 3 by 1 kernel size
net2.features[12] = nn.MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
num_ftrs = net2.classifier[6].in_features
net2.classifier[6] = nn.Linear(num_ftrs, my_num_classes)
#print(net2)
# Derniere couche
# (classifier): (6): Linear(in_features=4096, out_features=101, bias=True)
# (features): (12): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)

# modif net3, squeezenet1_0
net3.classifier[1] = nn.Conv2d(512, my_num_classes, kernel_size=(1,1), stride=(1,1))
#print(net3)
# Derniere couche
# (classifier): Sequential(
    #(0): Dropout(p=0.5, inplace=False)
    #(1): Conv2d(512, 101, kernel_size=(1, 1), stride=(1, 1))
    #(2): ReLU(inplace=True)
    #(3): AdaptiveAvgPool2d(output_size=(1, 1))

# modif net4, vgg16. Idem que AlexNet.
num_ftrs = net4.classifier[6].in_features
net4.classifier[6] = nn.Linear(num_ftrs, my_num_classes)
#print(net4)
# Derniere couche
# (6): Linear(in_features=4096, out_features=101, bias=True)

# modif net5, densenet161
# (classifier): Linear(in_features=2208, out_features=1000, bias=True)
num_ftrs = net5.classifier.in_features
net5.classifier = nn.Linear(num_ftrs, my_num_classes)
#print(net5)
# Derniere couche
# (classifier): Linear(in_features=2208, out_features=101, bias=True)

# modif net6, inception_v3
net6.aux_logits=False
#net6.AuxLogits.fc = nn.Linear(768, my_num_classes)
net6.fc = nn.Linear(2048, my_num_classes)

#net.Mixed_7a.branch3x3_2 = nn.Conv2d(192, 320, kernel_size=(1, 1), stride=(2, 2), padding=(1, 1), bias=False)
#net.Mixed_7a.branch7x7x3_4 = nn.Conv2d(192, 192, kernel_size=(1, 1), stride=(2, 2), padding=(1, 1), bias=False)
#net.Mixed_7b.branch3x3dbl_2 = nn.Conv2d(448, 384, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)
#net.Mixed_7c.branch3x3dbl_2 = nn.Conv2d(448, 384, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1), bias=False)

#net.Conv2d_1a_3x3 = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
#net.Conv2d_2a_3x3 = nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
#net.Conv2d_4a_3x3 = nn.Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

# Finalement, la solution est de desactiver les aux_logits pour faire marcher le reseau.
# https://stackoverflow.com/questions/53476305/attributeerror-tuple-object-has-no-attribute-log-softmax

#print(net6)
# input_size = 299 !!!
# OK! 

Chargement des images  Caltech101, environ 10000 images 300X200 pixels. Les objets sont rangés selon 101 catégories. 
http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz

Documentation pour ImageFolder: https://pytorch.org/vision/stable/datasets.html#imagefolder

D'abord, j'avais chargé le dataset total, avec son loader et j'ai visualisé une image pour verifier que les images sont bien chargées -> toutes les lignes commentées dans le code.

Ici, image_size = 224! Donc, pour tous les modeles sauf Inception_v3.
Les ensembles d'apprentissage (train) et de validation (test) sont d'une proportion 90%:10%. Nous avons 8683 images en total.

In [52]:
batchsize = 32
image_size = 224
proportion = 0.1
#image_size = 299
# Inveption_v3 requires input_size = 299!!!

# Transformations à appliquer pour l'ensemble d'apprentissage
transformTrain = transforms.Compose(
    [transforms.Resize((image_size, image_size)),
     transforms.CenterCrop(image_size),
     transforms.ToTensor(),
     transforms.RandomRotation(10),
     transforms.RandomHorizontalFlip(), 
     #transforms.RandomResizedCrop(32),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
     ])
# Pour l'ensemble de validation   
transformTest = transforms.Compose(
    [transforms.Resize((image_size, image_size)),
     transforms.CenterCrop(image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])



train_dataset = datasets.ImageFolder('drive/MyDrive/data/101_ObjectCategories', transform=transformTrain)
#loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize, pin_memory=True, shuffle=True)
# Recuperation du premier batch
#images,labs=iter(loader).next()
#print("Dimension d'un mini-batch de loader: ", images.size())
#img = images[0]
#print(" img.size() = ", img.size())
#print(" img.dim() = ", img.dim())
#print(" img.size()[0] = ", img.size()[0])
#Visualisation des images
#fig=plt.figure(figsize=(8, 8))
tr = transforms.ToPILImage()
#img0 = tr(img)
#plt.imshow(np.array(img0))

# On charge avec la transformation differente pour le test
test_dataset = datasets.ImageFolder('drive/MyDrive/data/101_ObjectCategories', transform=transformTest)

num_raw = len(train_dataset)
indices = list(range(num_raw))
split = int(np.floor(proportion * num_raw)) # 10% pour l'ensemble de validation
print(" length raw dataset = ", num_raw)
#print(" indices: ", indices)
print(" split = ", split)

 length raw dataset =  8683
 split =  868


Maintenant, je vais effectuer le split en train et test plusieurs fois (N_splits). Pour cela on crée une première boucle. Dans cette boucle il faudra integrer la boucle sur nets=[net1, net2, net3, net4, net5].

On ne touche pas encore le modele Inception_v3!

In [53]:
# On veut effectuer N fois le split en train et test
N_splits = 2

for s in range(N_splits):
  print(" ************************** Split en train et test #", s)
  # On melange les indices des images
  #np.random.seed(13)
  np.random.shuffle(indices)
  #print(" indices: ", indices)

  # division en 2 ensembles, l'indice = split est la frontiere entre train et test
  train_indices =  indices[split:]
  print(" Train length = ", len(train_indices))
  test_indices = indices[:split]
  print(" Test length = ", len(test_indices))

  train_subset = Subset(train_dataset, train_indices)
  trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batchsize, pin_memory=True, shuffle=True)
  train_size = len(train_subset)
  print(" Train size: ", train_size)

  # on refait la même procedure que avant pour créer le loader avec les indices des images
  test_subset = Subset(test_dataset, test_indices)
  testloader = torch.utils.data.DataLoader(test_subset, batch_size=batchsize, pin_memory=True, shuffle=True)
  test_size = len(test_subset)
  print(" Test size: ", test_size)

  # Recuperation du premier batch
  bx, by = iter(trainloader).next()
  print("Dimension d'un mini-batch de train_loader: ", bx.size())
  img_bx = bx[0]
  print(" img_bx.size() = ", img_bx.size())
  print(" img_bx.dim() = ", img_bx.dim())
  print(" img_bx.size()[0] = ", img_bx.size()[0])
  #Visualisation des images
  #fig=plt.figure(figsize=(8, 8))
  #img1 = tr(img_bx)
  #plt.imshow(np.array(img1))

  # Recuperation du premier batch
  bx, by = iter(testloader).next()
  print("Dimension d'un mini-batch de test_loader: ", bx.size())
  img_bx = bx[0]
  print(" img_bx.size() = ", img_bx.size())
  print(" img_bx.dim() = ", img_bx.dim())
  print(" img_bx.size()[0] = ", img_bx.size()[0])

  #Visualisation des images
  #fig=plt.figure(figsize=(8, 8))
  #img1 = tr(img_bx)
  #plt.imshow(np.array(img1))

  # Integration de la boucle sur les 5 premiers modeles
  i=1
  for net in nets:
    net.to(device)
    params_to_update = []
    for name,param in net.named_parameters():
      if param.requires_grad == True:
        params_to_update.append(param)
    optimizer = optim.SGD(params_to_update, lr=0.01, momentum=0.9)
    #tbw = SummaryWriter("trainCaltech101_net")
    print(" Started to train net", i)
    train()
    print(" Finished to train net", i)
    i=i+1
    

 ************************** Split en train et test # 0
 Train length =  7815
 Test length =  868
 Train size:  7815
 Test size:  868
Dimension d'un mini-batch de train_loader:  torch.Size([32, 3, 224, 224])
 img_bx.size() =  torch.Size([3, 224, 224])
 img_bx.dim() =  3
 img_bx.size()[0] =  3
Dimension d'un mini-batch de test_loader:  torch.Size([32, 3, 224, 224])
 img_bx.size() =  torch.Size([3, 224, 224])
 img_bx.dim() =  3
 img_bx.size()[0] =  3
 Started to train net 1



train loss= 0 1.3234032426561628



test loss= 0 0.6145170533231327
Accuracy in test 84.21658986175115



train loss= 1 0.5019195317993967



test loss= 1 0.5342769920825958
Accuracy in test 86.05990783410138


...


test loss= 0 0.2555018747225404
Accuracy in test 93.66359447004608



train loss= 1 0.23489353121543416



test loss= 1 0.2838316099452121
Accuracy in test 91.01382488479263



train loss= 2 0.20106755644083024



test loss= 2 0.27004742841901525
Accuracy in test 92.05069124423963
 Finished to train net 1


J'ai essayé d'utiliser TensorBoard, mais pas vraiment reussi à avoir un résultat util.

In [54]:
#Spécial notebook, les commandes suivantes permettent de lancer tensorboard
#%load_ext tensorboard
#%tensorboard --logdir logs
#from torch.utils.tensorboard import SummaryWriter
# Spécial notebook : pour avoir les courbes qui s'affichent dans le notebook, 
# sinon aller à l'adresse web local indiquée lors du lancement de tensorboard
#from tensorboard import notebook
#notebook.display()

Pour le taux moyen de bonne classification, il aurait fallu enregistrer la valeur "accuracy" pour chaque split, et pour chaque epoque du split. Ici, j'utilise Nb_epochs = 3, N_splits =2, ce qui donne 6 iterations en total. Donc, le taux moyen est la somme des accuracy_k, divisée par 6.

Je n'ai pas fait cela dans le code par faute de temps.

Avec les valeurs, recuperées à la main, et pour 3 epoques * 2 splits, cela donne:

AccuracyResnet_mean = ;

AccuracyAlexnet_mean = ;

AccuracySqueezenet_mean = ;

AccuracyVGG16_mean = ;

AccuracyDensenet_mean = ;


Je m'occupe maintenat du modele Inception_v3 separement.

In [57]:
# Pour le modele Inception_v3
image_size_iv3 = 299

#Spécial notebook, les commandes suivantes permettent de lancer tensorboard
#%load_ext tensorboard
#%tensorboard --logdir logs
#from torch.utils.tensorboard import SummaryWriter
# Spécial notebook : pour avoir les courbes qui s'affichent dans le notebook, 
# sinon aller à l'adresse web local indiquée lors du lancement de tensorboard
#from tensorboard import notebook
#notebook.display()

transformTrain_iv3 = transforms.Compose(
    [transforms.Resize((image_size_iv3, image_size_iv3)),
     transforms.CenterCrop(image_size_iv3),
     transforms.ToTensor(),
     transforms.RandomRotation(10),
     transforms.RandomHorizontalFlip(), 
     #transforms.RandomResizedCrop(32),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
     ])
transformTest_iv3 = transforms.Compose(
    [transforms.Resize((image_size_iv3, image_size_iv3)),
     transforms.CenterCrop(image_size_iv3),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = datasets.ImageFolder('drive/MyDrive/data/101_ObjectCategories', transform=transformTrain_iv3)
test_dataset = datasets.ImageFolder('drive/MyDrive/data/101_ObjectCategories', transform=transformTest_iv3)
print("Inception_v3 image_size is: ", image_size_iv3)

num_raw = len(train_dataset)
indices = list(range(num_raw))
split = int(np.floor(proportion * num_raw))

# On fait le split en train et test N fois
N_splits = 2
for s in range(N_splits):
  print(" ************************** Split en train et test #", s)
  np.random.shuffle(indices)

  train_indices =  indices[split:]
  print(" Train length IV3 = ", len(train_indices))
  test_indices = indices[:split]
  print(" Test length IV3 = ", len(test_indices))

  train_subset = Subset(train_dataset, train_indices)
  trainloader = torch.utils.data.DataLoader(train_subset, batch_size=batchsize, pin_memory=True, shuffle=True)
  train_size = len(train_subset)
  print(" Train size IV3: ", train_size)

  test_subset = Subset(test_dataset, test_indices)
  testloader = torch.utils.data.DataLoader(test_subset, batch_size=batchsize, pin_memory=True, shuffle=True)
  test_size = len(test_subset)
  print(" Test size IV3: ", test_size)

  for net in nets2:
    net.to(device)
    params_to_update = []
    for name,param in net.named_parameters():
      if param.requires_grad == True:
        params_to_update.append(param)
    optimizer = optim.SGD(params_to_update, lr=0.01, momentum=0.9)
    #tbw = SummaryWriter("trainCaltech101_net")
    print(" ****************** Started to train net Inception_v3")
    train()
    print(" *********** Finished to train net Inception_v3! ************************************ ")

Inception_v3 image_size is:  299
 ************************** Split en train et test # 0
 Train length IV3 =  7815
 Test length IV3 =  868
 Train size IV3:  7815
 Test size IV3:  868
 ****************** Started to train net Inception_v3



train loss= 0 1.919591804669828



test loss= 0 0.9733648278883525
Accuracy in test 83.17972350230414



train loss= 1 0.6846937320670303



test loss= 1 0.5632271968892643
Accuracy in test 87.32718894009217


...


test loss= 0 0.4082633719912597
Accuracy in test 90.2073732718894



train loss= 1 0.3666276162376209



test loss= 1 0.40283241282616344
Accuracy in test 90.89861751152074



train loss= 2 0.3267149336484014



test loss= 2 0.3542474117130041
Accuracy in test 90.66820276497695
 *********** Finished to train net Inception_v3! ************************************ 


Pour le taux moyen de bonne classification, comme avant, j'utilise Nb_epochs = 3, N_splits =2, ce qui donne 6 iterations en total.

Avec les valeurs, recuperées à la main, cela donne:

AccuracyInceptionV3_mean = ;